In [4]:
from bs4 import BeautifulSoup
import requests
import json
from flask import Flask, request
headers={
    "User-Agent":"Mozilla/5.0 (compatible; Googlebot/2.1; +https://www.google.com/bot.html)"
}        

def extract_product_details(div, featuredictionary):
    pdetails=div.find_all('div',class_="product-details")
    for product in pdetails:
        #product quantity available
        instock=product.find_all('span',class_="pdp-availability")
        #this shortcut
        #print(product.find_all('strong',class_="green"))
        for i in instock:
            stocka=i.find_all('strong',class_="green")
            #print(stocka[0].text.strip().replace('\n',' '))
            featuredictionary['in stock']=stocka[0].text.strip().replace('\n',' ')
        #Min Quantity per consignee:
        minq=product.find_all('span',class_="moq_data")
        #print(minq[0].text)
        featuredictionary['minimum quantity']=minq[0].text.strip()
        
        productid=product.find_all('span',class_="item_sku")
        #print(productid[0].text)
        featuredictionary['product id']=productid[0].text.strip()
        
        origin=product.find_all('span',class_="origin_country_data")
        #print(origin[0].text)
        featuredictionary['origin']=origin[0].text.strip()
        
        price=product.find_all('span',class_="m-w")
        #print(price[0].text)
        featuredictionary['price']=price[0].text.strip()
        
        mit=product.find_all('span',class_="mii_percentage_data")
        #print(mit[0].text)
        featuredictionary['mit']=mit[0].text.strip()
        
        pricefor=product.find_all('div',class_="pdp-qty-message")
        #print(pricefor[0].text.strip().replace('\n',' '))
        featuredictionary['pricefor']=pricefor[0].text.strip().replace('\n',' ')
        

def extract_images(div, featuredictionary):
    images = div.find_all('img')
    for image in images:
        im = image.get('src')
        featuredictionary['img'] = im

def extract_features(soup, featuredictionary):
    features = soup.body.find_all('div', id='feature_groups')
    row = features[0].find_all('tr')
    
    for r in row:
        data = r.find_all('td')
        featuredictionary[data[0].text.strip()] = data[1].text.strip()




#gemscraper
def gem_scraper(url):
    
    response = requests.get(url,headers=headers)
    featuredictionary={}
    itemdetails=""
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        itemdetails = soup.body.find_all('div', class_='page_bg')
        #function to extract title
        for div in itemdetails:
            titles=div.find_all('h1',class_='like-h3')
            for title in titles:
                    title_text = title.contents[0].strip() if title.contents else ""
                    brand=title.find_all('span',class_="brand-name")
                    model=title.find_all('span',class_="model")
                    featuredictionary['name']=title_text
                    featuredictionary['brand']=brand[0].text.strip()
                    featuredictionary['model']=model[0].text.strip()
            #product details
            extract_product_details(div,featuredictionary)
                
            sellerdetails=div.find_all('div',class_='seller-details')
            #print (sellerdetails)
            for detail in sellerdetails:
                sellertype=detail.find_all('span',class_='sold_as_summary')
                #print(sellertype[0].text.strip())
                featuredictionary['sellertype']=sellertype[0].text.strip()
                
                verificationstatus=detail.find_all('div', class_="seller-verified-status")
                #print(verificationstatus[0].text.strip())
                featuredictionary['verificationstatus']=verificationstatus[0].text.strip()
                
                rating=detail.find_all('span',class_="badge")
                #print(rating[0].text.strip())
                featuredictionary['rating']=rating[0].text.strip()
                
                sellerslink=detail.find_all('a')
                #print(sellerslink)
                #print('https://mkp.gem.gov.in/'+sellerslink[0].get('href'))
                featuredictionary['sellerslink']='https://mkp.gem.gov.in/'+sellerslink[0].get('href')
                
            extract_images(div,featuredictionary)
        extract_features(soup,featuredictionary)
        json_data = json.dumps(featuredictionary)
        return(json_data)
    else:
        print("Failed to fetch the webpage")

In [5]:
gem_scraper("https://mkp.gem.gov.in/executive-table-v2/executive-table/p-5116877-66941997902-cat.html#variant_id=5116877-66941997902")

'{"name": "DEVENT COLLECTION Executive Table with One side pedestal unit", "brand": "DEVENT COLLECTIONR", "model": "(GRP-111)", "in stock": "1200             In Stock", "minimum quantity": "1", "product id": "5116877-66941997902", "origin": "India", "price": "\\u20b913,305.00", "mit": "100%", "pricefor": "Price For :    1   pieces", "sellertype": "Resellers", "verificationstatus": "OEM verified Reseller", "rating": "4.0 - 4.49", "sellerslink": "https://mkp.gem.gov.in//executive-table-v2/executive-table/p-5116877-66941997902-cat/all_sellers.html", "img": "https://assets-mkpbg.gem.gov.in/img/othe/7317288/00/62/g1.jfif.18eb2e0062.999x120x120.jpg", "Structure /Frame of table": "Gable end and modesty panel", "Frame type": "Freestanding", "Accessories /Optional requrement for table": "Internet point", "The type of Storage provided": "One side pedestal unit", "Keyboard tray": "Provided", "Footrest": "Not Provided", "Mode of supply": "Knocked Down To Be Assembled At Consignee Site By The Selle